## 🔄 Cross-Attention (Encoder-Decoder Attention) — Detaylı Açıklama

### 1️⃣ Amaç ve Kullanım Alanı
Cross-Attention, **transformer tabanlı Encoder-Decoder modellerinde** kritik bir mekanizmadır.  
- Encoder, girdiyi anlamlı bir temsil (embedding) haline getirir.  
- Decoder, çıktıyı üretirken **encoder’in çıktısına bakarak** bağlam bilgisini alır.  
- Örnek: Çeviri modellerinde, decoder bir kelime üretirken hangi kaynak kelimelere dikkat edeceğini öğrenir.


### 2️⃣ Girdi ve Çıkışlar
**Girdiler:**
- `x_dec` : Decoder inputları → Query (Q) için kullanılır  
- `x_enc` : Encoder outputları → Key (K) ve Value (V) için kullanılır  
- `mask_enc` : Encoder maskesi (padding veya attention mask)  

**Çıktılar:**
- `out` : Decoder token’larının bağlam ile zenginleştirilmiş temsili  
- `attn` : (batch, num_heads, L_dec, L_enc) → hangi decoder token’ının hangi encoder token’ına dikkat ettiğini gösterir

### 3️⃣ Adım Adım İşleyiş

#### 🔹 3.1 Query, Key, Value Hesaplama
- Decoder token’ları `x_dec` → Q  
- Encoder token’ları `x_enc` → K ve V  

\[
Q = x_{dec} W_Q, \quad K = x_{enc} W_K, \quad V = x_{enc} W_V
\]

- Eğer multi-head ise, her Q/K/V tensoru `(batch, num_heads, seq_len, head_dim)` şeklinde yeniden şekillendirilir.  


#### 🔹 3.2 Skor Hesabı
- Decoder token’ları, encoder token’larıyla **benzerlik skorları** üzerinden ilişkilendirilir:

\[
\text{scores} = \frac{Q K^T}{\sqrt{d_k}}
\]

- `d_k` → head boyutu, ölçekleme faktörü olarak kullanılır.  
- Amaç: Skorların büyüklüğünü normalize edip softmax sonrası stabil dağılım sağlamak.



#### 🔹 3.3 Mask Uygulama
- Encoder inputlarında padding veya mask gerektiren durumlarda, skorlar mask ile `-inf` doldurularak softmax sonrası dikkate alınmaz.  
- Bu sayede decoder, gereksiz veya geçersiz token’lara dikkat etmez.


#### 🔹 3.4 Softmax ve Ağırlıklı Toplama
- Softmax uygulanarak skorlar olasılığa çevrilir: hangi decoder token’ının hangi encoder token’ına ne kadar dikkat edeceği belirlenir.
- Ardından **Value (V)** ile çarpılarak decoder token’ları için **bağlam (context)** oluşturulur:

\[
\text{context} = \text{Softmax(scores)} \cdot V
\]

- Bu adım, decoder token’ının **encoder bağlamını öğrenmesini** sağlar.


#### 🔹 3.5 Head Birleştirme ve Çıkış
- Multi-head attention: Tüm head’ler concat edilir ve `W_O` lineer katmanı ile projekte edilir.  
- Residual bağlantı eklenir ve LayerNorm uygulanır:

\[
\text{out} = \text{LayerNorm}(x_{dec} + \text{Dropout}(\text{context}))
\]

- Böylece hem giriş bilgisini korumuş oluruz hem de bağlam ile zenginleştirilmiş bir temsil elde ederiz.



### 4️⃣ Özellikler ve Avantajlar
- **Bağlam entegrasyonu:** Decoder, yalnızca kendi geçmiş token’larına değil, encoder’ın tamamına bakabilir.  
- **Multi-Head:** Farklı “bakış açıları” ile ilişkiler paralel olarak öğrenilir.  
- **Masking:** Hem padding hem de geleceğe bakma (causal) maskeleri ile esnek kullanım.  
- **Residual + LayerNorm:** Stabil ve derinleştirilmiş öğrenme olanağı sağlar.



### 5️⃣ Özet Akış Şeması
1. Decoder input → Q  
2. Encoder output → K, V  
3. Skor hesaplama: QKᵀ / √d_k  
4. Mask uygulama (varsa)  
5. Softmax → Attention ağırlıkları  
6. Ağırlıklı toplam: attn × V → context  
7. Multi-head birleştirme + Lineer çıkış  
8. Residual + LayerNorm → son çıktı
